### Import the required libraries and modules:

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
import numpy as np

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
%cd /content/gdrive/MyDrive/medical_image_processing/

Mounted at /content/gdrive
/content/gdrive/MyDrive/medical_image_processing


In [ ]:
from fundal_image.utils import create_dir, get_filenames_sorted
from patches import generate_patches_images_only, reconstruct_patches

In [ ]:
loss_str = "dice_bce"
with_augmentation = True
stride=512
seed = 42

aug_string = ""
if with_augmentation:
    aug_string = "_hvr"
root_path = f"/content/datasets/hrf_512_{stride}_{loss_str}{aug_string}/"
images_path = f"{root_path}/images/"
gt_path = f"{root_path}/groundtruth"

create_dir(images_path)
create_dir(gt_path)

In [ ]:
!cp /content/gdrive/MyDrive/medical_image_processing/fundal_image/datasets/HRF/images/*.* {root_path}/images
!cp /content/gdrive/MyDrive/medical_image_processing/fundal_image/datasets/HRF/groundtruth/*.* {root_path}/groundtruth/



In [ ]:
!ls {images_path}

01_dr.JPG  03_g.jpg   05_h.jpg	 08_dr.JPG  10_g.jpg   12_h.jpg   15_dr.JPG
01_g.jpg   03_h.jpg   06_dr.JPG  08_g.jpg   10_h.jpg   13_dr.JPG  15_g.jpg
01_h.jpg   04_dr.JPG  06_g.jpg	 08_h.jpg   11_dr.JPG  13_g.jpg   15_h.jpg
02_dr.JPG  04_g.jpg   06_h.jpg	 09_dr.JPG  11_g.jpg   13_h.jpg
02_g.jpg   04_h.jpg   07_dr.JPG  09_g.jpg   11_h.jpg   14_dr.JPG
02_h.jpg   05_dr.JPG  07_g.jpg	 09_h.jpg   12_dr.JPG  14_g.jpg
03_dr.JPG  05_g.jpg   07_h.jpg	 10_dr.JPG  12_g.jpg   14_h.jpg


In [ ]:

images_files, gt_files = get_filenames_sorted(images_path, gt_path)
print(len(images_files), len(gt_files))


45 45


In [ ]:
label = [_xtrain.rsplit("/", 1)[1].rsplit(".", 1)[0].rsplit("_", 1)[1] for _xtrain in images_files] 
print(label)

['dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h']


In [ ]:
# train_test_split
xtrain = []
xtest = []
test_x = None
if test_x is None or len(test_x) == 0:
    print("Creating train test split")
    from sklearn.model_selection import train_test_split
    label = [_xtrain.rsplit("/", 1)[1].rsplit(".", 1)[0].rsplit("_", 1)[1] for _xtrain in images_files] 
    xtrain, xtest, ytrain, ytest = train_test_split(images_files, gt_files, stratify=label,  test_size=0.4, random_state=seed)
else:
    # copy if there are any images without data_augmentation
    xtrain = [f for f in images_files if f.rsplit("/", 1)[1].rsplit(".", 1)[0] not in test_x]
    xtest = [f for f in images_files if f.rsplit("/", 1)[1].rsplit(".", 1)[0]  in test_x]

    # copy augmented images
    for f in images_files:
        name = f.rsplit("/", 1)[1].rsplit(".", 1)[0]
        if name.rsplit("_", 1)[1].isnumeric():
            if name.rsplit("_", 1)[0] in test_x:
                xtest.append(f)
            else:
                xtrain.append(f)




Creating train test split


In [ ]:
print(len(xtrain), len(xtest))


27 18


In [ ]:
from albumentations import HorizontalFlip, VerticalFlip, Rotate
def augment_data(images, masks, save_path, augment=True):
    H = 2336
    W = 3504

    for idx, (x, y) in tqdm(enumerate(zip(images, masks)), total=len(images)):
        """ Extracting names """
        name = x.rsplit("/", 1)[1].rsplit(".", 1)[0]
        print(name)
        """ Reading image and mask """
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = cv2.imread(y, cv2.IMREAD_GRAYSCALE)

        if augment == True:
            aug = HorizontalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x1 = augmented["image"]
            y1 = augmented["mask"]

            aug = VerticalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x2 = augmented["image"]
            y2 = augmented["mask"]

            aug = Rotate(limit=45, p=1.0)
            augmented = aug(image=x, mask=y)
            x3 = augmented["image"]
            y3 = augmented["mask"]

            X = [x, x1, x2, x3]
            Y = [y, y1, y2, y3]

            X = [x, x1, x2, x3]
            Y = [y, y1, y2, y3]

        else:
            X = [x]
            Y = [y]

        index = 0
        for i, m in zip(X, Y):
            i = cv2.resize(i, (W, H))
            m = cv2.resize(m, (W, H))

            if len(X) == 1:
                tmp_image_name = f"{name}.png"
                tmp_mask_name = f"{name}.png"
            else:
                tmp_image_name = f"{name}_{index}.png"
                tmp_mask_name = f"{name}_{index}.png"

            image_path = os.path.join(save_path, "image", tmp_image_name)
            mask_path = os.path.join(save_path, "mask", tmp_mask_name)

            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)

            index += 1

In [ ]:
create_dir(f"{root_path}/augmented/train/image/")
create_dir(f"{root_path}/augmented/train/mask/")

In [ ]:
from tqdm import tqdm

In [ ]:
augment_data(xtrain, ytrain, f"{root_path}/augmented/train/", True )

  0%|          | 0/27 [00:00<?, ?it/s]

13_g


  4%|▎         | 1/27 [00:01<00:42,  1.65s/it]

11_g


  7%|▋         | 2/27 [00:03<00:41,  1.65s/it]

...

 93%|█████████▎| 25/27 [00:41<00:03,  1.66s/it]

05_h


 96%|█████████▋| 26/27 [00:42<00:01,  1.65s/it]

03_dr


100%|██████████| 27/27 [00:44<00:00,  1.65s/it]


In [ ]:
xtrain, ytrain = get_filenames_sorted(f"{root_path}/augmented/train/image", f"{root_path}/augmented/train/mask")

In [ ]:
print(len(xtrain), len(ytrain))

108 108


In [ ]:
image_sizes = {"HRF": (2336, 3504), "drive": (584, 565), "chase_db": (960, 999)}
patch_size = {"HRF": (512, 512)}
stride = 512

dataset_name = "HRF"
num_pixels_to_add = [0, 0]
if (image_sizes["HRF"][0] % patch_size["HRF"][0]) > 0:
    num_pixels_to_add[0] = ((image_sizes["HRF"][0] // patch_size["HRF"][0]) + 1) * patch_size["HRF"][0]
    num_pixels_to_add[1] = ((image_sizes["HRF"][1] // patch_size["HRF"][1]) + 1) * patch_size["HRF"][1]

split_name = "split_2"
data_path = root_path
splits =["train","test"]


split = "train"
#for split in splits:
image_patches_path = f"{data_path}/{split_name}/{split}/image/"
create_dir(image_patches_path)
gt_patches_path = f"{data_path}/{split_name}/{split}/mask/"
create_dir(gt_patches_path)
im_patches_dict_train, metadata_train_df, gt_patches_dict_train = generate_patches_images_only(xtrain,
                                                                image_patches_path,
                                                                patch_size=patch_size["HRF"],
                                                                stride=stride,
                                                                zero_padding_size=num_pixels_to_add,
                                                                gt_files=ytrain,
                                                                save_path_gt=gt_patches_path)
metadata_train_df.to_csv(f"{data_path}/{split_name}/image_patches_512_512_{split}.csv", index=False)



Generating patches for 108 images


100%|██████████| 108/108 [01:13<00:00,  1.48it/s]


In [ ]:
split = "test"
#for split in splits:
image_patches_path = f"{data_path}/{split_name}/{split}/image/"
create_dir(image_patches_path)
gt_patches_path = f"{data_path}/{split_name}/{split}/mask/"
create_dir(gt_patches_path)
im_patches_dict_test, metadata_test_df, gt_patches_dict_test = generate_patches_images_only(xtest,
                                                                image_patches_path,
                                                                patch_size=patch_size["HRF"],
                                                                stride=stride,
                                                                zero_padding_size=num_pixels_to_add,
                                                                gt_files=ytest,
                                                                save_path_gt=gt_patches_path)
metadata_test_df.to_csv(f"{data_path}/{split_name}/image_patches_512_512_{split}.csv", index=False)


Generating patches for 18 images


100%|██████████| 18/18 [00:10<00:00,  1.67it/s]


In [ ]:
metadata_test_df.shape

(630, 7)

In [ ]:
metadata_train_df.head()

,image_filename,row,col,patch_height,patch_width,stride,num_fg_pixels
0,/content/datasets/hrf_512_512_dice_bce_hvr//au...,0,0,512,512,512,118326
1,/content/datasets/hrf_512_512_dice_bce_hvr//au...,0,512,512,512,512,200890
2,/content/datasets/hrf_512_512_dice_bce_hvr//au...,0,1024,512,512,512,204800
3,/content/datasets/hrf_512_512_dice_bce_hvr//au...,0,1536,512,512,512,204800
4,/content/datasets/hrf_512_512_dice_bce_hvr//au...,0,2048,512,512,512,204800


In [ ]:
import glob
split = "test"
image_patches_path = f"{data_path}/{split_name}/{split}/image/"

reconstructed_original_path = f"{data_path}/{split_name}/reconstructed/{split}/image"
create_dir(reconstructed_original_path)
reconstruct_patches(im_patches_dict_test,
                    metadata_test_df,
                    reconstructed_original_path
                    )


In [ ]:
reconstructed_gt_path = f"{data_path}/{split_name}/reconstructed/{split}/mask"

create_dir(reconstructed_gt_path)
reconstruct_patches(gt_patches_dict_test,
                    metadata_test_df,
                    reconstructed_gt_path,
                    )


In [ ]:
print([k for k in im_patches_dict_test.keys()])

['11_dr', '05_dr', '07_dr', '01_h', '15_g', '06_h', '15_h', '04_dr', '10_h', '06_g', '02_g', '05_g', '13_dr', '09_h', '03_g', '08_dr', '08_g', '12_h']


### Construct the Model:

In [ ]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model

input_shape = (512, 512, 3)
model = build_unet(input_shape)
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 512, 512, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                              

### Adding the Metrics:

In [ ]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

def DiceBCELoss(y_true, y_pred):
    
    inputs = tf.keras.layers.Flatten()(y_true)
    targets = tf.keras.layers.Flatten()(y_pred)

    intersection = tf.reduce_sum(inputs * targets)
    dice_loss = 1 - (2.*intersection + smooth)/(tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)
    BCE = tf.keras.losses.BinaryCrossentropy()
    Dice_BCE = BCE(inputs, targets) + dice_loss

    return Dice_BCE

def bce(y_true, y_pred):
    
    inputs = tf.keras.layers.Flatten()(y_true)
    targets = tf.keras.layers.Flatten()(y_pred)
    BCE = tf.keras.losses.BinaryCrossentropy()
    Dice_BCE = BCE(inputs, targets)

    return Dice_BCE



### Training the Model:

In [ ]:
if loss_str == "bce":
    loss = bce
elif loss_str == "dice_bce":
    loss = DiceBCELoss
elif loss_str == "dice_loss":
    loss = dice_loss
else:
    raise Exception("Invalid value for loss_str")
    


In [ ]:
loss

<function __main__.DiceBCELoss>

In [ ]:
H = 512
W = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.png")))
    y = sorted(glob(os.path.join(path, "mask", "*.png")))
    return x, y

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)              ## (512, 512, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

In [ ]:
from keras.metrics import AUC

In [ ]:
!ls

build			  hrf_bv_results	 README.md
classifier_data		  hrf_bv_results_sept_8  setup.py
dist			  logs			 tests.py
fundal_image		  metrics.py		 trained_models
fundal_image.egg-info	  patches.py		 trained_models_sept_8
hrf_512_512_dice_bce_hvr  __pycache__


In [ ]:
model_dir = f"{root_path}/trained_models/"
create_dir(model_dir)
%cd /content/datasets

/content/datasets


In [ ]:
num_epochs_completed = 0

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import CustomObjectScope


In [ ]:
from glob import glob

In [ ]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory to save files """
create_dir(model_dir)

""" Hyperparameters """
batch_size = 8
lr = 1e-4
num_epochs = 50

last_model_path = os.path.join(f"{model_dir}", f"model_{num_epochs_completed}.h5")

model_path = os.path.join(f"{model_dir}", "model.h5")

csv_path = os.path.join(f"{model_dir}", "data.csv")

""" Dataset """
dataset_path = f"{data_path}/{split_name}"
train_path = os.path.join(dataset_path, "train")
valid_path = os.path.join(dataset_path, "test")

train_x, train_y = load_data(train_path)
train_x, train_y = shuffling(train_x, train_y)
valid_x, valid_y = load_data(valid_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")

train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

train_steps = len(train_x)//batch_size
valid_setps = len(valid_x)//batch_size

if len(train_x) % batch_size != 0:
    train_steps += 1
if len(valid_x) % batch_size != 0:
    valid_setps += 1

""" Model """
model = build_unet((H, W, 3))
# with CustomObjectScope({"DiceBCELoss":DiceBCELoss, "dice_coef":dice_coef, "iou":iou}):
#     model = load_model(last_model_path)
model.compile(loss=loss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision(), AUC()])

callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True, monitor="val_dice_coef", mode="max"),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-6, verbose=1),
    CSVLogger(csv_path),
    TensorBoard(),
    EarlyStopping(monitor="val_dice_coef", patience=10, restore_best_weights=False, mode="max")
]

model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    steps_per_epoch=train_steps,
    validation_steps=valid_setps,
    callbacks=callbacks
)

Train: 3780 - 3780
Valid: 630 - 630
Epoch 1/50
473/473 [==============================] - ETA: 0s - loss: 0.6475 - dice_coef: 0.5202 - iou: 0.3621 - recall: 0.7565 - precision: 0.6442 - auc: 0.9424
Epoch 00001: val_dice_coef improved from -inf to 0.48786, saving model to /content/datasets/hrf_512_512_dice_bce_hvr//trained_models/model.h5
473/473 [==============================] - 469s 956ms/step - loss: 0.6475 - dice_coef: 0.5202 - iou: 0.3621 - recall: 0.7565 - precision: 0.6442 - auc: 0.9424 - val_loss: 0.6541 - val_dice_coef: 0.4879 - val_iou: 0.3272 - val_recall: 0.5405 - val_precision: 0.7800 - val_auc: 0.8931 - lr: 1.0000e-04
Epoch 2/50
473/473 [==============================] - ETA: 0s - loss: 0.4011 - dice_coef: 0.6943 - iou: 0.5338 - recall: 0.8111 - precision: 0.7796 - auc: 0.9709
Epoch 00002: val_dice_coef improved from 0.48786 to 0.73480, saving model to /content/datasets/hrf_512_512_dice_bce_hvr//trained_models/model.h5
473/473 [==============================] - 450s 952ms

### Evaluating the Model:

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
from tensorflow.keras.models import load_model

In [ ]:
H = 512
W = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    return ori_x, x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.int32)
    return ori_x, x

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.png")))
    y = sorted(glob(os.path.join(path, "mask", "*.png")))
    return x, y

def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)

In [ ]:
import glob
supported_extensions = ["png", "jpg", "tif", "jpeg", "gif", "bmp"]

def image_files(path, file_prefix=None):
    all_image_files = []
    # print("path", path)
    # print("file_prefix", file_prefix)
    if file_prefix is None:
        file_names = list(glob.glob(path + "/*.*" ))
    else:
        prefix___ = path + "/" + file_prefix + ".*"
        #print("prefix", prefix___)
        file_names = list(glob.glob(prefix___))

    # print("filenames", file_names)

    for filename in file_names:
        ext = filename.rsplit(".", 1)[1]
        if ext.lower() in supported_extensions:
            all_image_files.append(filename)
    # print("All Image files", all_image_files)
    return sorted(all_image_files)


In [ ]:
# from glob import glob
# def get_filenames_sorted(image_path, gt_path=None):
#     image_file_names = image_files(image_path)
#     gt_file_names = None
#     if gt_path is not None:
#         # Get the test files also
#         gt_file_names = sorted(image_files(gt_path))
#     return image_file_names, gt_file_names

In [ ]:
results_path = f"{root_path}/results/"
prediction_path = f"{root_path}/predictions/"
prediction_np_path = f"{root_path}/predictions_np/"
create_dir(prediction_path)
create_dir(prediction_np_path)
create_dir(results_path)

In [ ]:
results_path

'/content/datasets/hrf_512_512_dice_bce_hvr//results/'

In [ ]:
from glob import glob

In [ ]:
from copy import deepcopy
from sklearn.metrics import roc_auc_score
# prediction_path = "predictions"

""" Save the results in this folder """
# create_dir("results_sunil_test")
# create_dir(prediction_path)

""" Load the model """
# with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
model_path = os.path.join(f"{model_dir}", "model.h5")

model = load_model(model_path, compile=False)

""" Load the dataset """
dataset_path = os.path.join(f"{data_path}/{split_name}", "test")
test_x, test_y = load_data(dataset_path)

""" Make the prediction and calculate the metrics values """
SCORE = []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracting name """
    name = x.split("/")[-1].split(".")[0]

    """ Read the image and mask """
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)
    y = y.astype(np.int)

    """ Prediction """
    y_pred = model.predict(np.expand_dims(x, axis=0))[0]
    np.save(f"{prediction_np_path}/{name}.npy", y_pred)
    y_prob = deepcopy(y_pred).flatten()
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(np.int32)
    y_pred = np.squeeze(y_pred, axis=-1)

    """ Saving the images """
    save_image_path = f"{results_path}/{name}.png"
#    print(ori_x.shape, ori_y.shape, y_pred.shape)
    save_results(ori_x, ori_y, y_pred, save_image_path)
    cv2.imwrite(f"{prediction_path}/{name}.png", y_pred * 255)
    

    """ Flatten the array """
    y = y.flatten()
    y_pred = y_pred.flatten()

    """ Calculate the metrics """
    acc_value = accuracy_score(y, y_pred)
    f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
    jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
    recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
    precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
    if np.sum(y) == 0 or np.sum(y) == H * W:
        auc_value = 1
    else:
        auc_value = roc_auc_score(y, y_prob)

    SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, auc_value])

score = [s[1:] for s in SCORE]
score = np.mean(score, axis=0)
print(f"Accuracy: {score[0]:0.5f}")
print(f"F1: {score[1]:0.5f}")
print(f"Jaccard: {score[2]:0.5f}")
print(f"Recall: {score[3]:0.5f}")
print(f"Precision: {score[4]:0.5f}")
print(f"AUC: {score[5]:0.5f}")

""" Saving """
df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "AUC"])
df.to_csv(f"{results_path}/score.csv")

 14%|█▍        | 91/630 [00:47<04:27,  2.02it/s]/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_clas

Accuracy: 0.97555
F1: 0.77107
Jaccard: 0.64549
Recall: 0.77623
Precision: 0.78241
AUC: 0.98786


In [ ]:
model_dir

'/content/datasets/hrf_512_512_dice_bce_hvr//trained_models/'

# Reconstruct predicted image

In [ ]:
prediction_np_path

'/content/datasets/hrf_512_512_dice_bce_hvr//predictions_np/'

In [ ]:
from glob import glob

In [ ]:
# Create patches dictionary
predicted_patches_dict = dict()
for file in glob(f"{prediction_np_path}/*.npy"):
    print("file", file)
    patch = np.load(file)
    fname_without_ext = file.rsplit("/", 1)[1].rsplit(".", 1)[0]
    fname_without_ext = fname_without_ext[1:]
    patch_number = int(fname_without_ext.rsplit("_", 1)[1])
    image_file = fname_without_ext.rsplit("_", 1)[0]
    print(image_file, patch_number )

    if image_file in predicted_patches_dict.keys():
        predicted_patches_dict[image_file][patch_number] = np.load(file)
    else:
        patches = [None] * 140
        patches[patch_number] = np.load(file)
        predicted_patches_dict[image_file] = patches

        




file /content/datasets/hrf_512_512_dice_bce_hvr//predictions_np/_06_g_33.npy
06_g 33
file /content/datasets/hrf_512_512_dice_bce_hvr//predictions_np/_07_dr_25.npy
07_dr 25
file /content/datasets/hrf_512_512_dice_bce_hvr//predictions_np/_08_dr_16.npy
08_dr 16
file /content/datasets/hrf_512_512_dice_bce_hvr//predictions_np/_11_dr_9.npy
11_dr 9
file /content/datasets/hrf_512_512_dice_bce_hvr//predictions_np/_07_dr_26.npy
07_dr 26
file /content/datasets/hrf_512_512_dice_bce_hvr//predictions_np/_15_h_1.npy
15_h 1
file /content/datasets/hrf_512_512_dice_bce_hvr//predictions_np/_15_g_32.npy
15_g 32
file /content/datasets/hrf_512_512_dice_bce_hvr//predictions_np/_01_h_0.npy
01_h 0
file /content/datasets/hrf_512_512_dice_bce_hvr//predictions_np/_13_dr_6.npy
13_dr 6
file /content/datasets/hrf_512_512_dice_bce_hvr//predictions_np/_15_g_13.npy
15_g 13
file /content/datasets/hrf_512_512_dice_bce_hvr//predictions_np/_09_h_12.npy
09_h 12
file /content/datasets/hrf_512_512_dice_bce_hvr//predictions_np

In [ ]:
reconstructed_predictions_path = f"{data_path}/{split_name}/reconstructed/{split}/predictions_reconstructed"
create_dir(reconstructed_predictions_path)
reconstruct_patches(predicted_patches_dict,
                    metadata_test_df,
                    reconstructed_predictions_path,
                    mfactor=255
                    )


In [ ]:
!pwd

/content/datasets


In [ ]:
def read_mask(path):
    print(path)
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
#    x = x.astype(np.int32)
    return ori_x, x

In [ ]:
def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((2560, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    print(f"Saving image of size {cat_images.shape} to {save_image_path}")
    cv2.imwrite(save_image_path, cat_images)

In [ ]:
results_path_reconstructed = f"{root_path}/results_reconstructed/"
create_dir(results_path_reconstructed)

In [ ]:
from glob import glob

In [ ]:
from copy import deepcopy
from sklearn.metrics import roc_auc_score
# prediction_path = "predictions"

""" Load the model """
# with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
model_path = os.path.join(f"{model_dir}", "model.h5")
model = load_model(model_path, compile=False)

""" Load the dataset """
# dataset_path = os.path.join(f"{data_path}/{split_name}", "test")

test_x, test_y = load_data(f"{data_path}/{split_name}/reconstructed/test/")

""" Make the prediction and calculate the metrics values """
SCORE = []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracting name """
    name = x.split("/")[-1].split(".")[0]

    """ Read the image and mask """
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)
    y = y.astype(np.int)
    print(name)

#     """ Prediction """
    #y_pred = model.predict(np.expand_dims(x, axis=0))[0]

    orig_y_pred, y_pred = read_mask(f"{data_path}/{split_name}/reconstructed/test/predictions_reconstructed/{name}.png")
    print(y_pred.shape, np.max(y_pred), np.min(y_pred))
    print(y_pred.shape, np.max(orig_y_pred), np.min(orig_y_pred))
    y_prob = deepcopy(y_pred).flatten()
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(np.int32)

    """ Saving the images """
    save_image_path = f"{results_path_reconstructed}/{name}.png"
    print(ori_x.shape, ori_y.shape,y_pred.shape)
    save_results(ori_x, ori_y, y_pred, save_image_path)

    
#    cv2.imwrite(f"{prediction_path}/{name}.png", y_pred * 255)
    

    """ Flatten the array """
    y = y.flatten()
    y_pred = y_pred.flatten()

    """ Calculate the metrics """
    acc_value = accuracy_score(y, y_pred)
    f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
    jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
    recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
    precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
    if np.sum(y) == 0 or np.sum(y) == H * W:
        auc_value = 1
    else:
        auc_value = roc_auc_score(y, y_prob)

    SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, auc_value])

score = [s[1:] for s in SCORE]
score = np.mean(score, axis=0)
print(f"Accuracy: {score[0]:0.5f}")
print(f"F1: {score[1]:0.5f}")
print(f"Jaccard: {score[2]:0.5f}")
print(f"Recall: {score[3]:0.5f}")
print(f"Precision: {score[4]:0.5f}")
print(f"AUC: {score[5]:0.5f}")

""" Saving """
df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "AUC"])
df.to_csv(f"{results_path_reconstructed}/score_full.csv")

  0%|          | 0/18 [00:00<?, ?it/s]

/content/datasets/hrf_512_512_dice_bce_hvr//split_2/reconstructed/test/mask/01_h.png
01_h
/content/datasets/hrf_512_512_dice_bce_hvr//split_2/reconstructed/test/predictions_reconstructed/01_h.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_dice_bce_hvr//results_reconstructed//01_h.png


  6%|▌         | 1/18 [00:17<04:58, 17.56s/it]

/content/datasets/hrf_512_512_dice_bce_hvr//split_2/reconstructed/test/mask/02_g.png
02_g
/content/datasets/hrf_512_512_dice_bce_hvr//split_2/reconstructed/test/predictions_reconstructed/02_g.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_dice_bce_hvr//results_reconstructed//02_g.png


 11%|█         | 2/18 [00:34<04:36, 17.29s/it]

...

/content/datasets/hrf_512_512_dice_bce_hvr//split_2/reconstructed/test/mask/15_g.png
15_g
/content/datasets/hrf_512_512_dice_bce_hvr//split_2/reconstructed/test/predictions_reconstructed/15_g.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_dice_bce_hvr//results_reconstructed//15_g.png


 94%|█████████▍| 17/18 [04:54<00:17, 17.41s/it]

/content/datasets/hrf_512_512_dice_bce_hvr//split_2/reconstructed/test/mask/15_h.png
15_h
/content/datasets/hrf_512_512_dice_bce_hvr//split_2/reconstructed/test/predictions_reconstructed/15_h.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_dice_bce_hvr//results_reconstructed//15_h.png


100%|██████████| 18/18 [05:11<00:00, 17.31s/it]

Accuracy: 0.97555
F1: 0.81889
Jaccard: 0.69443
Recall: 0.82406
Precision: 0.82062
AUC: 0.98395


In [ ]:
from copy import deepcopy
from sklearn.metrics import roc_auc_score
# prediction_path = "predictions"

""" Load the model """
# with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
model_path = os.path.join(f"{model_dir}", "model.h5")
model = load_model(model_path, compile=False)

""" Load the dataset """
# dataset_path = os.path.join(f"{data_path}/{split_name}", "test")

test_x, test_y = load_data(f"{data_path}/{split_name}/reconstructed/test/")


""" Make the prediction and calculate the metrics values """
SCORE = []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracting name """
    name = x.split("/")[-1].split(".")[0]

    """ Read the image and mask """
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)
    y = y.astype(np.int)
    print(name)

#     """ Prediction """
    #y_pred = model.predict(np.expand_dims(x, axis=0))[0]

    orig_y_pred, y_pred = read_mask(f"{data_path}/{split_name}/reconstructed/test/predictions_reconstructed/{name}.png")
    print(y_pred.shape, np.max(y_pred), np.min(y_pred))
    print(y_pred.shape, np.max(orig_y_pred), np.min(orig_y_pred))
    y_prob = deepcopy(y_pred).flatten()
    y_pred = y_pred > 0.6
    y_pred = y_pred.astype(np.int32)

    """ Saving the images """
    save_image_path = f"{results_path_reconstructed}/{name}.png"
    print(ori_x.shape, ori_y.shape,y_pred.shape)
    save_results(ori_x, ori_y, y_pred, save_image_path)

    
#    cv2.imwrite(f"{prediction_path}/{name}.png", y_pred * 255)
    

    """ Flatten the array """
    y = y.flatten()
    y_pred = y_pred.flatten()

    """ Calculate the metrics """
    acc_value = accuracy_score(y, y_pred)
    f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
    jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
    recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
    precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
    if np.sum(y) == 0 or np.sum(y) == H * W:
        auc_value = 1
    else:
        auc_value = roc_auc_score(y, y_prob)

    SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, auc_value])

score = [s[1:] for s in SCORE]
score = np.mean(score, axis=0)
print(f"Accuracy: {score[0]:0.5f}")
print(f"F1: {score[1]:0.5f}")
print(f"Jaccard: {score[2]:0.5f}")
print(f"Recall: {score[3]:0.5f}")
print(f"Precision: {score[4]:0.5f}")
print(f"AUC: {score[5]:0.5f}")

""" Saving """
df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "AUC"])
df.to_csv(f"{results_path_reconstructed}/score_full.csv")

  0%|          | 0/18 [00:00<?, ?it/s]

/content/datasets/hrf_512_512_dice_bce_hvr//split_2/reconstructed/test/mask/01_h.png
01_h
/content/datasets/hrf_512_512_dice_bce_hvr//split_2/reconstructed/test/predictions_reconstructed/01_h.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_dice_bce_hvr//results_reconstructed//01_h.png


  6%|▌         | 1/18 [00:17<05:05, 17.97s/it]

/content/datasets/hrf_512_512_dice_bce_hvr//split_2/reconstructed/test/mask/02_g.png
02_g
/content/datasets/hrf_512_512_dice_bce_hvr//split_2/reconstructed/test/predictions_reconstructed/02_g.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_dice_bce_hvr//results_reconstructed//02_g.png


 11%|█         | 2/18 [00:35<04:46, 17.92s/it]

...

/content/datasets/hrf_512_512_dice_bce_hvr//split_2/reconstructed/test/mask/15_g.png
15_g
/content/datasets/hrf_512_512_dice_bce_hvr//split_2/reconstructed/test/predictions_reconstructed/15_g.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_dice_bce_hvr//results_reconstructed//15_g.png


 94%|█████████▍| 17/18 [04:57<00:17, 17.46s/it]

/content/datasets/hrf_512_512_dice_bce_hvr//split_2/reconstructed/test/mask/15_h.png
15_h
/content/datasets/hrf_512_512_dice_bce_hvr//split_2/reconstructed/test/predictions_reconstructed/15_h.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_dice_bce_hvr//results_reconstructed//15_h.png


100%|██████████| 18/18 [05:14<00:00, 17.49s/it]

Accuracy: 0.97576
F1: 0.81817
Jaccard: 0.69322
Recall: 0.81237
Precision: 0.83114
AUC: 0.98395


In [ ]:
from copy import deepcopy
from sklearn.metrics import roc_auc_score
# prediction_path = "predictions"

""" Load the model """
# with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
model_path = os.path.join(f"{model_dir}", "model.h5")
model = load_model(model_path, compile=False)

""" Load the dataset """
# dataset_path = os.path.join(f"{data_path}/{split_name}", "test")

test_x, test_y = load_data(f"{data_path}/{split_name}/reconstructed/test/")


""" Make the prediction and calculate the metrics values """
SCORE = []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracting name """
    name = x.split("/")[-1].split(".")[0]

    """ Read the image and mask """
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)
    y = y.astype(np.int)
    print(name)

#     """ Prediction """
    #y_pred = model.predict(np.expand_dims(x, axis=0))[0]

    orig_y_pred, y_pred = read_mask(f"{data_path}/{split_name}/reconstructed/test/predictions_reconstructed/{name}.png")
    print(y_pred.shape, np.max(y_pred), np.min(y_pred))
    print(y_pred.shape, np.max(orig_y_pred), np.min(orig_y_pred))
    y_prob = deepcopy(y_pred).flatten()
    y_pred = y_pred > 0.4
    y_pred = y_pred.astype(np.int32)

    """ Saving the images """
    save_image_path = f"{results_path_reconstructed}/{name}.png"
    print(ori_x.shape, ori_y.shape,y_pred.shape)
    save_results(ori_x, ori_y, y_pred, save_image_path)

    
    # cv2.imwrite(f"{prediction_path}/{name}.png", y_pred * 255)
    

    """ Flatten the array """
    y = y.flatten()
    y_pred = y_pred.flatten()

    """ Calculate the metrics """
    acc_value = accuracy_score(y, y_pred)
    f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
    jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
    recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
    precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
    if np.sum(y) == 0 or np.sum(y) == H * W:
        auc_value = 1
    else:
        auc_value = roc_auc_score(y, y_prob)

    SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, auc_value])

score = [s[1:] for s in SCORE]
score = np.mean(score, axis=0)
print(f"Accuracy: {score[0]:0.5f}")
print(f"F1: {score[1]:0.5f}")
print(f"Jaccard: {score[2]:0.5f}")
print(f"Recall: {score[3]:0.5f}")
print(f"Precision: {score[4]:0.5f}")
print(f"AUC: {score[5]:0.5f}")

""" Saving """
df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "AUC"])
df.to_csv(f"{results_path_reconstructed}/score_full.csv")

  0%|          | 0/18 [00:00<?, ?it/s]

/content/datasets/hrf_512_512_dice_bce_hvr//split_2/reconstructed/test/mask/01_h.png
01_h
/content/datasets/hrf_512_512_dice_bce_hvr//split_2/reconstructed/test/predictions_reconstructed/01_h.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_dice_bce_hvr//results_reconstructed//01_h.png


  6%|▌         | 1/18 [00:17<04:56, 17.47s/it]

/content/datasets/hrf_512_512_dice_bce_hvr//split_2/reconstructed/test/mask/02_g.png
02_g
/content/datasets/hrf_512_512_dice_bce_hvr//split_2/reconstructed/test/predictions_reconstructed/02_g.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_dice_bce_hvr//results_reconstructed//02_g.png


 11%|█         | 2/18 [00:34<04:40, 17.50s/it]

...

/content/datasets/hrf_512_512_dice_bce_hvr//split_2/reconstructed/test/mask/15_g.png
15_g
/content/datasets/hrf_512_512_dice_bce_hvr//split_2/reconstructed/test/predictions_reconstructed/15_g.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_dice_bce_hvr//results_reconstructed//15_g.png


 94%|█████████▍| 17/18 [04:57<00:17, 17.57s/it]

/content/datasets/hrf_512_512_dice_bce_hvr//split_2/reconstructed/test/mask/15_h.png
15_h
/content/datasets/hrf_512_512_dice_bce_hvr//split_2/reconstructed/test/predictions_reconstructed/15_h.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_dice_bce_hvr//results_reconstructed//15_h.png


100%|██████████| 18/18 [05:14<00:00, 17.50s/it]

Accuracy: 0.97526
F1: 0.81897
Jaccard: 0.69472
Recall: 0.83463
Precision: 0.81049
AUC: 0.98395


In [ ]:
history_df = pd.read_csv(csv_path)

In [ ]:
history_df.shape

(21, 14)

In [ ]:
history_df.head(50)

,epoch,auc,dice_coef,iou,loss,lr,precision,recall,val_auc,val_dice_coef,val_iou,val_loss,val_precision,val_recall
0,0,0.942392,0.520174,0.362140,0.647506,0.000100,0.644214,0.756511,0.893073,0.487865,0.327175,0.654105,0.779991,0.540500
1,1,0.970865,0.694259,0.533782,0.401051,0.000100,0.779572,0.811136,0.975947,0.734801,0.582983,0.347977,0.820193,0.803577
2,2,0.977677,0.745462,0.595622,0.340090,0.000100,0.796279,0.816774,0.982243,0.764852,0.621153,0.312034,0.829907,0.804474
3,3,0.981337,0.768164,0.624796,0.313927,0.000100,0.803994,0.819251,0.983471,0.781640,0.643379,0.293119,0.826080,0.813917
4,4,0.981592,0.781110,0.641926,0.298875,0.000100,0.809217,0.822113,0.981259,0.788777,0.653059,0.286482,0.822515,0.819468
5,5,0.980988,0.788829,0.652328,0.289839,0.000100,0.812964,0.824220,0.978686,0.791805,0.657115,0.283722,0.831394,0.809701
6,6,0.980133,0.791392,0.655836,0.287460,0.000100,0.814017,0.822850,0.975769,0.793935,0.659977,0.283674,0.834267,0.805502
7,7,0.980612,0.796906,0.663393,0.280488,0.000100,0.817161,0.826636,0.978275,0.796692,0.663756,0.278809,0.833344,0.810109
8,8,0.981502,0.802749,0.671439,0.272715,0.000100,0.821143,0.831483,0.977048,0.797725,0.665188,0.278865,0.830687,0.812217
9,9,0.982020,0.807802,0.678490,0.266265,0.000100,0.824760,0.835698,0.977820,0.801071,0.669820,0.276188,0.825289,0.819459


In [ ]:
results_path

'/content/datasets/hrf_512_512_dice_bce_hvr//results/'

In [ ]:
!ls 

hrf_512_512_dice_bce_hvr  logs


In [ ]:
!mv hrf_512_512_dice_bce_hvr hrf_512_512_dice_bce_hvr_2

In [ ]:
!zip -r hrf_512_512_dice_bce_hvr_2.zip  hrf_512_512_dice_bce_hvr_2/

Streaming output truncated to the last 5000 lines.
  adding: hrf_512_512_dice_bce_hvr_2/split_2/train/mask/_01_dr_1_8.png (deflated 31%)
  adding: hrf_512_512_dice_bce_hvr_2/split_2/train/mask/_07_h_3_25.png (deflated 25%)
  adding: hrf_512_512_dice_bce_hvr_2/split_2/train/mask/_11_g_3_15.png (deflated 26%)
  adding: hrf_512_512_dice_bce_hvr_2/split_2/train/mask/_10_g_3_8.png (deflated 25%)
  adding: hrf_512_512_dice_bce_hvr_2/split_2/train/mask/_14_dr_3_28.png (deflated 45%)
  adding: hrf_512_512_dice_bce_hvr_2/split_2/train/mask/_10_dr_1_17.png (deflated 32%)
  adding: hrf_512_512_dice_bce_hvr_2/split_2/train/mask/_02_h_0_27.png (deflated 31%)
  adding: hrf_512_512_dice_bce_hvr_2/split_2/train/mask/_10_g_1_1.png (deflated 35%)
  adding: hrf_512_512_dice_bce_hvr_2/split_2/train/mask/_14_h_0_13.png (deflated 33%)
  adding: hrf_512_512_dice_bce_hvr_2/split_2/train/mask/_09_g_0_14.png (deflated 34%)
  adding: hrf_512_512_dice_bce_hvr_2/split_2/train/mask/_10_dr_1_9.png (deflated 30%)
  a

In [ ]:
!ls -ltrh


total 2.8G
drwxr-xr-x  4 root root 4.0K Feb  3 05:25 logs
drwxr-xr-x 11 root root 4.0K Feb  3 08:18 hrf_512_512_dice_bce_hvr_2
-rw-r--r--  1 root root 2.8G Feb  3 08:55 hrf_512_512_dice_bce_hvr_2.zip


In [ ]:
!cp hrf_512_512_dice_bce_hvr_2.zip /content/gdrive/MyDrive/Blood_Vessel_Results/hrf_patches/

In [ ]:
# %cd /content/gdrive/MyDrive/Blood_Vessel_Results/hrf_patches/

In [ ]:
!pwd

/content/datasets


In [ ]:
!ls

hrf_512_512_dice_bce_hvr_2  hrf_512_512_dice_bce_hvr_2.zip  logs


In [ ]:
drive.flush_and_unmount()